In [16]:
# ============================================================================
# CELL 1: IMPORT THƯ VIỆN VÀ THIẾT LẬP
# ============================================================================

print("🔧 Đang thiết lập môi trường SQLite...")

import sqlite3
import time
import random
import json
from datetime import datetime

# Tạo database trong memory
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

print("✅ Đã thiết lập SQLite in-memory database!")

🔧 Đang thiết lập môi trường SQLite...
✅ Đã thiết lập SQLite in-memory database!


In [17]:
# ============================================================================
# CELL 2: KHỞI TẠO CÁC BẢNG
# ============================================================================

print("=" * 70)
print("KHỞI TẠO CÁC BẢNG TRONG SQLITE")
print("=" * 70)

# ===== PHƯƠNG PHÁP 1: RBAC THÔNG THƯỜNG (Normalized) =====
print("\n[1] Tạo bảng RBAC Thông thường (Normalized)...")

# Bảng Users
cursor.execute("""
    CREATE TABLE traditional_users (
        user_id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE,
        department TEXT,
        group_name TEXT
    );
""")

# Bảng Roles
cursor.execute("""
    CREATE TABLE traditional_roles (
        role_id INTEGER PRIMARY KEY AUTOINCREMENT,
        role_name TEXT UNIQUE
    );
""")

# Bảng Permissions
cursor.execute("""
    CREATE TABLE traditional_permissions (
        permission_id INTEGER PRIMARY KEY AUTOINCREMENT,
        resource TEXT,
        action TEXT,
        company TEXT,
        branch TEXT
    );
""")

# Bảng User-Role (Many-to-Many)
cursor.execute("""
    CREATE TABLE traditional_user_roles (
        user_id INTEGER,
        role_id INTEGER,
        PRIMARY KEY (user_id, role_id),
        FOREIGN KEY (user_id) REFERENCES traditional_users(user_id),
        FOREIGN KEY (role_id) REFERENCES traditional_roles(role_id)
    );
""")

# Bảng Role-Permission (Many-to-Many)
cursor.execute("""
    CREATE TABLE traditional_role_permissions (
        role_id INTEGER,
        permission_id INTEGER,
        PRIMARY KEY (role_id, permission_id),
        FOREIGN KEY (role_id) REFERENCES traditional_roles(role_id),
        FOREIGN KEY (permission_id) REFERENCES traditional_permissions(permission_id)
    );
""")

print("   ✓ Tạo 5 bảng normalized thành công")

# ===== PHƯƠNG PHÁP 2: RBAC PHÂN CẤP (Hierarchical with JSON) =====
print("\n[2] Tạo bảng RBAC Phân cấp (Hierarchical)...")

cursor.execute("""
    CREATE TABLE hierarchical_subjects (
        row_key TEXT PRIMARY KEY,
        value TEXT  -- SQLite không có JSONB, dùng TEXT thay thế
    );
""")

cursor.execute("""
    CREATE TABLE hierarchical_objects (
        row_key TEXT PRIMARY KEY,
        value TEXT
    );
""")

cursor.execute("""
    CREATE TABLE hierarchical_access_policy (
        row_key TEXT PRIMARY KEY,
        value TEXT
    );
""")

print("   ✓ Tạo 3 bảng hierarchical thành công")

# Hàm helper để extract JSON (thay thế cho JSONB operators)
def json_extract(data, key):
    """Hàm trích xuất giá trị từ JSON string"""
    try:
        if data:
            data_dict = json.loads(data)
            return data_dict.get(key)
        return None
    except:
        return None

# Đăng ký hàm với SQLite
conn.create_function("json_extract", 2, json_extract)

print("   ✓ Đã đăng ký hàm json_extract")

conn.commit()
print("\n✓ Khởi tạo database hoàn tất!\n")

KHỞI TẠO CÁC BẢNG TRONG SQLITE

[1] Tạo bảng RBAC Thông thường (Normalized)...
   ✓ Tạo 5 bảng normalized thành công

[2] Tạo bảng RBAC Phân cấp (Hierarchical)...
   ✓ Tạo 3 bảng hierarchical thành công
   ✓ Đã đăng ký hàm json_extract

✓ Khởi tạo database hoàn tất!



In [18]:
# ============================================================================
# CELL 3: SINH DỮ LIỆU TEST (FIXED VERSION 2 - SỬA LỖI JSON_EXTRACT)
# ============================================================================

def generate_test_data(num_users=500, num_files=200):
    """Sinh dữ liệu test cho cả 2 phương pháp - FIXED VERSION 2"""
    
    print("=" * 70)
    print(f"SINH DỮ LIỆU TEST ({num_users} users, {num_files} files)")
    print("=" * 70)
    
    departments = ['IT', 'HR', 'Sales', 'Finance', 'Marketing']
    groups = ['dev', 'qa', 'managers', 'analysts', 'support']
    companies = ['ABC', 'XYZ', 'FinanceCorp']
    branches = ['engineering', 'research', 'operations', 'audit']
    actions = ['read', 'write', 'delete']
    
    # ===== DỮ LIỆU CHO RBAC THÔNG THƯỜNG =====
    print("\n[1] Sinh dữ liệu RBAC Thông thường...")
    start = time.time()
    
    # Insert Users
    for i in range(num_users):
        username = f"user_{i}"
        dept = random.choice(departments)
        group = random.choice(groups)
        cursor.execute(
            "INSERT INTO traditional_users (username, department, group_name) VALUES (?, ?, ?)",
            (username, dept, group)
        )
    
    # Insert Roles
    roles = ['admin', 'developer', 'viewer', 'editor', 'analyst']
    for role in roles:
        cursor.execute("INSERT INTO traditional_roles (role_name) VALUES (?)", (role,))
    
    # Insert Permissions
    for i in range(num_files):
        resource = f"file_{i}.pdf"
        action = random.choice(actions)
        company = random.choice(companies)
        branch = random.choice(branches)
        cursor.execute(
            "INSERT INTO traditional_permissions (resource, action, company, branch) VALUES (?, ?, ?, ?)",
            (resource, action, company, branch)
        )
    
    # Assign Users to Roles (random)
    for i in range(1, num_users + 1):
        role_id = random.randint(1, 5)
        try:
            cursor.execute(
                "INSERT INTO traditional_user_roles (user_id, role_id) VALUES (?, ?)",
                (i, role_id)
            )
        except:
            pass
    
    # Assign Roles to Permissions (random)
    for role_id in range(1, 6):
        num_perms = random.randint(50, 150)
        for _ in range(num_perms):
            perm_id = random.randint(1, num_files)
            try:
                cursor.execute(
                    "INSERT INTO traditional_role_permissions (role_id, permission_id) VALUES (?, ?)",
                    (role_id, perm_id)
                )
            except:
                pass
    
    trad_time = time.time() - start
    print(f"   ✓ Hoàn tất trong {trad_time:.2f}s")
    
    # ===== DỮ LIỆU CHO RBAC PHÂN CẤP - FIXED VERSION 2 =====
    print("\n[2] Sinh dữ liệu RBAC Phân cấp (FIXED VERSION 2)...")
    start = time.time()
    
    # Tạo danh sách user info để dùng cho policies
    user_info = []
    for i in range(num_users):
        username = f"user_{i}"
        dept = random.choice(departments)
        group = random.choice(groups)
        user_info.append({
            'username': username,
            'department': dept,
            'group_name': group
        })
        
        cursor.execute(
            "INSERT INTO hierarchical_subjects (row_key, value) VALUES (?, ?)",
            (f"user:{i}", json.dumps({"username": username, "department": dept, "group_name": group}))
        )
    
    # Insert Objects
    for i in range(num_files):
        filename = f"file_{i}.pdf"
        company = random.choice(companies)
        branch = random.choice(branches)
        cursor.execute(
            "INSERT INTO hierarchical_objects (row_key, value) VALUES (?, ?)",
            (f"file:{i}", json.dumps({"file_name": filename, "company": company, "branch": branch}))
        )
    
    # Insert Access Policies - ĐƠN GIẢN HÓA ĐỂ ĐẢM BẢO CÓ DỮ LIỆU
    policy_count = 0
    
    # Tạo policies với các điều kiện CỤ THỂ cho testing
    print("   👥 Tạo policies với điều kiện cụ thể cho testing...")
    
    # 1. Policies cho IT department + dev group + ABC company + engineering branch + read action
    num_target_policies = 50  # Đảm bảo có đủ dữ liệu để test
    for i in range(num_target_policies):
        file_idx = random.randint(0, num_files - 1)
        filename = f"file_{file_idx}.pdf"
        
        policy_data = {
            "username": f"user_{random.randint(0, num_users-1)}",
            "department": "IT",
            "group_name": "dev",
            "file_name": filename,
            "company": "ABC",
            "branch": "engineering",
            "action": "read",
            "allowed": True,
            "policy_type": "target_test"
        }
        
        cursor.execute(
            "INSERT INTO hierarchical_access_policy (row_key, value) VALUES (?, ?)",
            (f"target_policy:{policy_count}", json.dumps(policy_data))
        )
        policy_count += 1
    
    # 2. Policies thông thường (random)
    print("   🎲 Tạo policies ngẫu nhiên...")
    for i in range(num_users):
        username = f"user_{i}"
        dept = random.choice(departments)
        group = random.choice(groups)
        
        num_policies = random.randint(5, 10)
        for j in range(num_policies):
            file_idx = random.randint(0, num_files - 1)
            filename = f"file_{file_idx}.pdf"
            company = random.choice(companies)
            branch = random.choice(branches)
            action = random.choice(actions)
            
            policy_data = {
                "username": username,
                "department": dept,
                "group_name": group,
                "file_name": filename,
                "company": company,
                "branch": branch,
                "action": action,
                "allowed": True,
                "policy_type": "user_random"
            }
            
            cursor.execute(
                "INSERT INTO hierarchical_access_policy (row_key, value) VALUES (?, ?)",
                (f"random_policy:{policy_count}", json.dumps(policy_data))
            )
            policy_count += 1
    
    hier_time = time.time() - start
    print(f"   ✓ Hoàn tất trong {hier_time:.2f}s")
    print(f"   ✓ Tạo {policy_count} access policies")
    
    # KIỂM TRA DỮ LIỆU BẰNG CÁCH TRỰC TIẾP - FIXED
    print(f"\n   🔍 KIỂM TRA DỮ LIỆU (TRỰC TIẾP):")
    
    # Đếm bằng cách fetch all và filter thủ công
    cursor.execute("SELECT value FROM hierarchical_access_policy")
    all_policies = cursor.fetchall()
    
    # Đếm thủ công
    it_dev_count = 0
    abc_eng_count = 0
    read_count = 0
    target_count = 0
    
    for policy in all_policies:
        policy_data = json.loads(policy[0])
        
        if policy_data.get('department') == 'IT' and policy_data.get('group_name') == 'dev':
            it_dev_count += 1
        
        if policy_data.get('company') == 'ABC' and policy_data.get('branch') == 'engineering':
            abc_eng_count += 1
        
        if policy_data.get('action') == 'read':
            read_count += 1
            
        if policy_data.get('policy_type') == 'target_test':
            target_count += 1
    
    print(f"   - Policies IT+dev: {it_dev_count}")
    print(f"   - Policies ABC+engineering: {abc_eng_count}")
    print(f"   - Policies read action: {read_count}")
    print(f"   - Policies target test: {target_count}")
    
    # Hiển thị vài policy mẫu
    print(f"\n   📄 VÀI POLICY MẪU:")
    sample_policies = all_policies[:3]
    for i, policy in enumerate(sample_policies):
        policy_data = json.loads(policy[0])
        print(f"      {i+1}. {policy_data}")
    
    conn.commit()
    print("\n✓ Sinh dữ liệu hoàn tất!\n")

# Chạy lại sinh dữ liệu (đã fix hoàn toàn)
generate_test_data(num_users=500, num_files=200)

SINH DỮ LIỆU TEST (500 users, 200 files)

[1] Sinh dữ liệu RBAC Thông thường...
   ✓ Hoàn tất trong 0.00s

[2] Sinh dữ liệu RBAC Phân cấp (FIXED VERSION 2)...
   👥 Tạo policies với điều kiện cụ thể cho testing...
   🎲 Tạo policies ngẫu nhiên...
   ✓ Hoàn tất trong 0.02s
   ✓ Tạo 3805 access policies

   🔍 KIỂM TRA DỮ LIỆU (TRỰC TIẾP):
   - Policies IT+dev: 225
   - Policies ABC+engineering: 344
   - Policies read action: 1247
   - Policies target test: 50

   📄 VÀI POLICY MẪU:
      1. {'username': 'user_73', 'department': 'IT', 'group_name': 'dev', 'file_name': 'file_38.pdf', 'company': 'ABC', 'branch': 'engineering', 'action': 'read', 'allowed': True, 'policy_type': 'target_test'}
      2. {'username': 'user_462', 'department': 'IT', 'group_name': 'dev', 'file_name': 'file_23.pdf', 'company': 'ABC', 'branch': 'engineering', 'action': 'read', 'allowed': True, 'policy_type': 'target_test'}
      3. {'username': 'user_362', 'department': 'IT', 'group_name': 'dev', 'file_name': 'file_182

In [19]:
# ============================================================================
# CELL 4-6: CẬP NHẬT CÁC HÀM BENCHMARK
# ============================================================================

def benchmark_complex_query_fixed(num_runs=20):
    """So sánh truy vấn phức tạp - FIXED VERSION"""
    
    print("\n" + "=" * 70)
    print("TEST 3: COMPLEX QUERY (FIXED)")
    print("Câu hỏi: Users trong IT department, thuộc group dev,")
    print("         có quyền READ files của ABC company, branch engineering")
    print("=" * 70)
    
    # ===== TRADITIONAL RBAC =====
    print("\n[TRADITIONAL RBAC]")
    query_traditional = """
        SELECT DISTINCT u.username, p.resource
        FROM traditional_users u
        JOIN traditional_user_roles ur ON u.user_id = ur.user_id
        JOIN traditional_roles r ON ur.role_id = r.role_id
        JOIN traditional_role_permissions rp ON r.role_id = rp.role_id
        JOIN traditional_permissions p ON rp.permission_id = p.permission_id
        WHERE u.department = ?
          AND u.group_name = ?
          AND p.action = ?
          AND p.company = ?
          AND p.branch = ?;
    """
    
    times_trad = []
    results_trad = []
    for _ in range(num_runs):
        start = time.time()
        cursor.execute(query_traditional, ('IT', 'dev', 'read', 'ABC', 'engineering'))
        results = cursor.fetchall()
        times_trad.append(time.time() - start)
        if not results_trad:  # Chỉ lưu kết quả đầu tiên
            results_trad = results
    
    avg_trad = sum(times_trad) * 1000 / len(times_trad)
    print(f"📊 Kết quả:")
    print(f"   - Số records: {len(results_trad)}")
    print(f"   - Thời gian TB: {avg_trad:.2f}ms")
    
    # ===== HIERARCHICAL RBAC - SỬ DỤNG PHƯƠNG PHÁP THỦ CÔNG =====
    print("\n[HIERARCHICAL RBAC - Manual JSON Filtering]")
    
    times_hier = []
    results_hier = []
    
    for _ in range(num_runs):
        start = time.time()
        
        # Lấy tất cả policies và filter thủ công
        cursor.execute("SELECT value FROM hierarchical_access_policy")
        all_policies = cursor.fetchall()
        
        # Filter thủ công
        filtered_results = []
        for policy in all_policies:
            policy_data = json.loads(policy[0])
            if (policy_data.get('department') == 'IT' and 
                policy_data.get('group_name') == 'dev' and 
                policy_data.get('action') == 'read' and 
                policy_data.get('company') == 'ABC' and 
                policy_data.get('branch') == 'engineering' and
                policy_data.get('allowed', True)):
                
                filtered_results.append({
                    'user': policy_data.get('username', 'N/A'),
                    'file': policy_data.get('file_name')
                })
        
        times_hier.append(time.time() - start)
        if not results_hier:  # Chỉ lưu kết quả đầu tiên
            results_hier = filtered_results
    
    avg_hier = sum(times_hier) * 1000 / len(times_hier)
    print(f"📊 Kết quả:")
    print(f"   - Số records: {len(results_hier)}")
    print(f"   - Thời gian TB: {avg_hier:.2f}ms")
    
    # Hiển thị vài kết quả mẫu
    if results_hier:
        print(f"   - Ví dụ kết quả (3 đầu):")
        for i, result in enumerate(results_hier[:3]):
            print(f"      {i+1}. User: {result['user']}, File: {result['file']}")
    
    speedup = avg_trad / avg_hier if avg_hier > 0 else 0
    if speedup > 1:
        print(f"\n🏆 Hierarchical NHANH HƠN: {speedup:.2f}x")
    else:
        print(f"\n🏆 Traditional NHANH HƠN: {1/speedup:.2f}x")

# Chạy benchmark với phương pháp fixed
benchmark_complex_query_fixed(num_runs=20)


TEST 3: COMPLEX QUERY (FIXED)
Câu hỏi: Users trong IT department, thuộc group dev,
         có quyền READ files của ABC company, branch engineering

[TRADITIONAL RBAC]
📊 Kết quả:
   - Số records: 60
   - Thời gian TB: 3.29ms

[HIERARCHICAL RBAC - Manual JSON Filtering]
📊 Kết quả:
   - Số records: 54
   - Thời gian TB: 4.33ms
   - Ví dụ kết quả (3 đầu):
      1. User: user_73, File: file_38.pdf
      2. User: user_462, File: file_23.pdf
      3. User: user_362, File: file_182.pdf

🏆 Traditional NHANH HƠN: 1.32x


In [20]:
# ============================================================================
# CELL 7: PHÂN TÍCH DUNG LƯỢNG LƯU TRỮ
# ============================================================================

def print_storage_stats():
    """So sánh dung lượng lưu trữ"""
    
    print("\n" + "=" * 70)
    print("PHÂN TÍCH DUNG LƯỢNG LƯU TRỮ")
    print("=" * 70)
    
    # Traditional tables
    trad_tables = [
        'traditional_users',
        'traditional_roles',
        'traditional_permissions',
        'traditional_user_roles',
        'traditional_role_permissions'
    ]
    
    print("\n[TRADITIONAL RBAC]")
    trad_total = 0
    for table in trad_tables:
        cursor.execute(f"SELECT COUNT(*) FROM {table}")
        count = cursor.fetchone()[0]
        print(f"   {table:40s}: {count:6d} records")
    
    # Hierarchical tables
    hier_tables = [
        'hierarchical_subjects',
        'hierarchical_objects',
        'hierarchical_access_policy'
    ]
    
    print("\n[HIERARCHICAL RBAC]")
    for table in hier_tables:
        cursor.execute(f"SELECT COUNT(*) FROM {table}")
        count = cursor.fetchone()[0]
        print(f"   {table:40s}: {count:6d} records")
    
    print(f"\n🏆 KẾT LUẬN TỔNG QUAN:")
    print("   • Hierarchical RBAC sử dụng ít bảng hơn (3 vs 5)")
    print("   • Truy vấn đơn giản hơn với JSON extraction")
    print("   • Linh hoạt hơn trong việc thêm trường dữ liệu mới")

# Chạy phân tích storage
print_storage_stats()


PHÂN TÍCH DUNG LƯỢNG LƯU TRỮ

[TRADITIONAL RBAC]
   traditional_users                       :    500 records
   traditional_roles                       :      5 records
   traditional_permissions                 :    200 records
   traditional_user_roles                  :    500 records
   traditional_role_permissions            :    369 records

[HIERARCHICAL RBAC]
   hierarchical_subjects                   :    500 records
   hierarchical_objects                    :    200 records
   hierarchical_access_policy              :   3805 records

🏆 KẾT LUẬN TỔNG QUAN:
   • Hierarchical RBAC sử dụng ít bảng hơn (3 vs 5)
   • Truy vấn đơn giản hơn với JSON extraction
   • Linh hoạt hơn trong việc thêm trường dữ liệu mới
